In [1]:
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd

main = pd.read_pickle("clean_main.df")

In [2]:
main

,race_id,horse_no,horse_id,result,won,lengths_behind,horse_age,horse_country,horse_type,horse_rating,...,place_combination4,place_dividend1,place_dividend2,place_dividend3,place_dividend4,win_combination1,win_dividend1,win_combination2,win_dividend2,num_of_horses
0,0,1,3917,10,0,8.00,3,AUS,Gelding,60,...,NaN,36.5,25.5,18.0,NaN,8,121.0,NaN,NaN,14
1,0,2,2157,8,0,5.75,3,NZ,Gelding,60,...,NaN,36.5,25.5,18.0,NaN,8,121.0,NaN,NaN,14
2,0,3,858,7,0,4.75,3,NZ,Gelding,60,...,NaN,36.5,25.5,18.0,NaN,8,121.0,NaN,NaN,14
3,0,4,1853,9,0,6.25,3,SAF,Gelding,60,...,NaN,36.5,25.5,18.0,NaN,8,121.0,NaN,NaN,14
4,0,5,2796,6,0,3.75,3,GB,Gelding,60,...,NaN,36.5,25.5,18.0,NaN,8,121.0,NaN,NaN,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76113,6348,10,1238,6,0,1.25,5,AUS,Gelding,87,...,NaN,38.0,63.0,18.5,NaN,2,134.0,NaN,NaN,14
76114,6348,11,985,7,0,2.25,5,NZ,Gelding,84,...,NaN,38.0,63.0,18.5,NaN,2,134.0,NaN,NaN,14
76115,6348,12,3147,10,0,3.00,5,NZ,Gelding,83,...,NaN,38.0,63.0,18.5,NaN,2,134.0,NaN,NaN,14
76116,6348,13,1982,8,0,2.50,3,AUS,Gelding,82,...,NaN,38.0,63.0,18.5,NaN,2,134.0,NaN,NaN,14


In [3]:
# find and remove uncommon races from the data
print("Different tracks:")
print(main["venue"].value_counts())
print("\nDifferent classes:")
print(main["race_class"].value_counts())
print("\nDifferent distances:")
print(main["distance"].value_counts())

Different tracks:
ST    50355
HV    25763
Name: venue, dtype: int64

Different classes:
4     28984
3     24769
5     11127
2      7068
1      1917
11     1248
13     1005
Name: race_class, dtype: int64

Different distances:
1200    23940
1400    14406
1650    13034
1000     8238
1600     8043
1800     7092
2000     1365
Name: distance, dtype: int64


In [4]:
print("Distances at ST:")
print(main.loc[main.venue=="ST"]["distance"].value_counts())

print("\nDistances at HV:")
print(main.loc[main.venue=="HV"]["distance"].value_counts())

Distances at ST:
1200    14603
1400    14406
1600     8043
1000     4422
1800     4333
1650     3183
2000     1365
Name: distance, dtype: int64

Distances at HV:
1650    9851
1200    9337
1000    3816
1800    2759
Name: distance, dtype: int64


In [5]:
print("Classes at ST:")
print(main.loc[main.venue=="ST"]["race_class"].value_counts())

print("\nClasses at HV:")
print(main.loc[main.venue=="HV"]["race_class"].value_counts())

Classes at ST:
4     18417
3     15544
5      7042
2      5182
1      1917
11     1248
13     1005
Name: race_class, dtype: int64

Classes at HV:
4    10567
3     9225
5     4085
2     1886
Name: race_class, dtype: int64


In [6]:
# Remove:
# races at HV that have class 1, 13 or distance 2200, 1400, 1600
# races at ST that have class 12, 6, 0 or distance 2400, 2200

indexNames = main[( (main["venue"]=="HV") & 
                  (main["distance"].isin([2200,1400,1600]) | main["race_class"].isin([1, 13])) ) |
                  ( (main["venue"]=="ST") & 
                  (main["distance"].isin([2400, 2200]) | main["race_class"].isin([12, 6, 0])))].index
main.drop(indexNames, inplace=True)
main = main.reset_index(drop=True)
main.to_pickle("clean_main.df")

In [2]:
# Need to also deal with runners that did not finish the race (will assign them the average lengths behind
# for a dead last horse)
# Or we could just remove them from the running datasets

# First we count how many
len(main.loc[main["lengths_behind"]>=70])

100

In [3]:
# Not much, should just replace with average loser time
losers = main.loc[main["num_of_horses"]==main["result"]].copy()
# remove ones that will be replaced
dropIndexs = losers[losers["lengths_behind"]>=70].index
losers.drop(dropIndexs, inplace=True)
loser_lengths = np.mean(losers["lengths_behind"])

In [4]:
# Set all values above 70 to this value (seems not that bad of a loser time?)
badIndexs = main[main["lengths_behind"]>=70].index
main.at[badIndexs, "lengths_behind"] = loser_lengths

len(main.loc[main["lengths_behind"]>=70])

0

In [5]:
main = main.reset_index(drop=True)
main.to_pickle("clean_main.df")

In [6]:
len(main.loc[main["lengths_behind"]>=70])

0